## MODELO BERTO PARA NER

In [17]:
%pip install transformers torch pandas scikit-learn numpy datasets seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install -U transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [19]:
# --- Importaciones ---
import json
import os
import torch
import numpy as np
import pandas as pd # Añadido por si se usa para explorar datos
from datasets import Dataset, DatasetDict, Features, Value, ClassLabel, Sequence
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score
from seqeval.scheme import IOB2 # Usaremos el esquema IOB2 (similar a BIO)
import traceback # Para imprimir errores detallados

print("Librerías instaladas e importadas.")

Librerías instaladas e importadas.


In [ ]:
# --- Configuración ---
# Asegúrate de que la ruta sea correcta en Kaggle (usualmente /kaggle/input/nombre-dataset/archivo.jsonl)
ARCHIVO_DATASET_JSONL = '/kaggle/input/pln-segundo-momento/Data/dataset_ner.jsonl' # <-- CAMBIA ESTA RUTA

MODELO_BERT_NER = 'dccuchile/bert-base-spanish-wwm-uncased' # Modelo BETO (o BERTIN)

# Directorio de salida en Kaggle (siempre dentro de /kaggle/working/)
OUTPUT_DIR_NER = '/kaggle/working/modelo_ner_frases'

MAX_LEN_NER = 128 # Longitud máxima de secuencia para BERT
BATCH_SIZE_NER = 8 # Lotes más pequeños suelen ser mejores para NER
EPOCHS_NER = 5      # NER puede requerir más épocas que clasificación
LEARNING_RATE_NER = 3e-5
LABEL_ALL_SUBWORDS = False # Estrategia para etiquetar subpalabras (False es común)

# Crear directorio de salida si no existe
os.makedirs(OUTPUT_DIR_NER, exist_ok=True)

print(f"Archivo de datos: {ARCHIVO_DATASET_JSONL}")
print(f"Modelo base: {MODELO_BERT_NER}")
print(f"Directorio de salida: {OUTPUT_DIR_NER}")

In [ ]:
def cargar_datos_jsonl(ruta_archivo):
    """Carga datos desde un archivo JSON Lines."""
    datos = []
    print(f"Intentando cargar datos desde: {ruta_archivo}")
    if not os.path.exists(ruta_archivo):
        print(f"Error: El archivo {ruta_archivo} no existe.")
        # En Kaggle, puedes listar los archivos de entrada para depurar:
        print("Archivos en /kaggle/input/:", os.listdir('/kaggle/input/'))
        # Lista recursivamente para encontrar tu archivo si está en subdirectorios
        for dirname, _, filenames in os.walk('/kaggle/input'):
             for filename in filenames:
                 print(os.path.join(dirname, filename))
        return None # Retorna None para indicar fallo

    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            for i, linea in enumerate(f):
                try:
                    datos.append(json.loads(linea))
                except json.JSONDecodeError:
                    print(f"Advertencia: Omitiendo línea mal formada (Línea {i+1}): {linea.strip()}")
        print(f"Cargados {len(datos)} ejemplos desde {ruta_archivo}")
        if not datos:
            print("Advertencia: No se cargaron datos válidos del archivo.")
            return None
        return datos
    except Exception as e:
        print(f"Error inesperado al cargar {ruta_archivo}: {e}")
        return None

def crear_dataset_hf(datos_cargados):
    """Convierte los datos cargados a un Dataset de Hugging Face."""
    if not datos_cargados:
        print("Error: No hay datos cargados para crear el dataset.")
        return None, None

    # Extraer textos y entidades
    textos = [ejemplo['text'] for ejemplo in datos_cargados]
    ner_tags_list = []
    all_labels_set = set()

    print("Procesando entidades...")
    for idx, ejemplo in enumerate(datos_cargados):
        tags_ejemplo = []
        if 'entities' in ejemplo and ejemplo['entities']:
            # Asegurarse de que 'entities' es una lista
            if not isinstance(ejemplo['entities'], list):
                print(f"Advertencia: 'entities' no es una lista en el ejemplo {idx}. Omitiendo.")
                ner_tags_list.append([]) # Añadir lista vacía
                continue

            for entity_data in ejemplo['entities']:
                # Validar formato de cada entidad
                if not (isinstance(entity_data, list) and len(entity_data) == 3):
                     print(f"Advertencia: Formato de entidad inválido {entity_data} en ejemplo {idx}. Omitiendo.")
                     continue

                start, end, label = entity_data
                # Validación de índices y tipo de etiqueta
                if not (isinstance(start, int) and isinstance(end, int) and isinstance(label, str)):
                     print(f"Advertencia: Tipos de datos inválidos en entidad {entity_data} en ejemplo {idx}. Omitiendo.")
                     continue
                if start >= end or start < 0 or end > len(ejemplo['text']):
                     print(f"Advertencia: Índices de entidad inválidos {entity_data} en '{ejemplo['text'][:50]}...' (Ejemplo {idx}). Omitiendo.")
                     continue

                tags_ejemplo.append({'start': start, 'end': end, 'label': label})
                all_labels_set.add(label)
        ner_tags_list.append(tags_ejemplo)

    if not all_labels_set:
        print("Error: No se encontraron etiquetas de entidad válidas en los datos.")
        return None, None

    # Crear la estructura de Features
    label_list = sorted(list(all_labels_set))
    features = Features({
        'id': Value('string'),
        'text': Value('string'),
        'ner_tags': Sequence({
            'start': Value('int32'),
            'end': Value('int32'),
            'label': ClassLabel(names=label_list)
        })
    })

    # Crear el Dataset
    try:
      hf_dataset = Dataset.from_dict(
          {
              "id": [str(i) for i in range(len(textos))],
              "text": textos,
              "ner_tags": ner_tags_list,
          },
          features=features
      )
    except Exception as e:
        print(f"Error al crear el Dataset de Hugging Face: {e}")
        # Imprimir algunos datos para depurar
        print("Ejemplo de textos:", textos[:2])
        print("Ejemplo de ner_tags_list:", ner_tags_list[:2])
        print("Características intentadas:", features)
        return None, None


    print("\nDataset de Hugging Face creado exitosamente.")
    print("Características (Features):", hf_dataset.features)
    print("Etiquetas de entidad encontradas:", label_list)
    return hf_dataset, label_list

print("Funciones de carga y creación de dataset definidas.")

In [ ]:
# --- Cargar los datos ---
datos_originales = cargar_datos_jsonl(ARCHIVO_DATASET_JSONL)

# --- Crear el Dataset de Hugging Face ---
if datos_originales:
    dataset_hf, label_list = crear_dataset_hf(datos_originales)
else:
    dataset_hf = None
    label_list = None

# Verificar si el dataset se creó correctamente
if dataset_hf is None or label_list is None:
    print("\nError: No se pudo crear el dataset. Revisa los mensajes anteriores.")
    # Detener la ejecución si falla la carga/creación
    raise RuntimeError("Fallo en la carga o creación del dataset.")
else:
    print("\nDataset listo para el siguiente paso.")
    print("Primer ejemplo del dataset:")
    print(dataset_hf[0])

In [ ]:
if label_list:
    # Obtener las etiquetas BIO
    bio_label_list = ["O"] + [f"B-{lbl}" for lbl in label_list] + [f"I-{lbl}" for lbl in label_list]
    label2id = {label: i for i, label in enumerate(bio_label_list)}
    id2label = {i: label for i, label in enumerate(bio_label_list)}
    num_labels_ner = len(bio_label_list)

    print(f"\nNúmero total de etiquetas BIO: {num_labels_ner}")
    print("Mapeo label2id (BIO):", label2id)

    # Guardar los mapeos BIO
    map_file_path_ner = os.path.join(OUTPUT_DIR_NER, 'ner_label_mappings.json')
    try:
        with open(map_file_path_ner, 'w', encoding='utf-8') as f:
            json.dump({'label2id': label2id, 'id2label': id2label}, f, ensure_ascii=False, indent=4)
        print(f"Mapeos de etiquetas NER guardados en: {map_file_path_ner}")
    except Exception as e:
        print(f"Error al guardar mapeos de etiquetas: {e}")
else:
    print("Error: No hay lista de etiquetas disponible. No se pueden generar etiquetas BIO.")
    raise RuntimeError("Fallo al generar etiquetas BIO.")

In [ ]:
# --- Cargar Tokenizador ---
print(f"\nCargando tokenizador NER para: {MODELO_BERT_NER}")
try:
    tokenizer_ner = AutoTokenizer.from_pretrained(MODELO_BERT_NER, use_fast=True)
    print("Tokenizador cargado.")
except Exception as e:
    print(f"Error al cargar el tokenizador {MODELO_BERT_NER}: {e}")
    raise RuntimeError("Fallo al cargar tokenizador.")


# --- Función de Tokenización y Alineación (ACCESO Y LÓGICA B/I CORREGIDOS) ---
def tokenize_and_align_labels(examples):
    """
    Tokeniza texto y alinea etiquetas NER con los tokens/subtokens.
    Espera que examples['ner_tags'] sea List[Dict[str, List[int]]].
    Utiliza el acceso correcto: examples['ner_tags'][i]['key'][j].
    Contiene lógica B/I corregida.
    """
    # Tokenizar todos los textos del lote a la vez
    tokenized_inputs = tokenizer_ner(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN_NER,
        return_offsets_mapping=True,
        is_split_into_words=False
    )

    all_aligned_labels = [] # Lista para guardar las etiquetas alineadas de todo el lote

    # Iterar sobre cada ejemplo *dentro* del lote usando un índice
    num_examples_in_batch = len(examples["text"])
    # examples['ner_tags'] es List[Dict[str, List[int]]]
    for i in range(num_examples_in_batch):

        # --- RECONSTRUIR la lista de entidades para el ejemplo 'i' ---
        # *** USANDO EL ACCESO CORRECTO examples['ner_tags'][i]['key'] ***
        try:
            # Acceder al diccionario del ejemplo i-th
            ner_tags_dict_for_example = examples['ner_tags'][i]

            # Acceder a las listas *dentro* de ese diccionario
            starts_list = ner_tags_dict_for_example.get('start', []) # List[int]
            ends_list = ner_tags_dict_for_example.get('end', [])     # List[int]
            labels_list = ner_tags_dict_for_example.get('label', []) # List[int] (IDs ClassLabel)
            num_entities_in_example = len(starts_list)

            # Validar que todas las listas tengan la misma longitud
            if not (len(starts_list) == len(ends_list) == len(labels_list)):
                 print(f"Advertencia: Longitudes inconsistentes en índice {i}. Start:{len(starts_list)}, End:{len(ends_list)}, Label:{len(labels_list)}. Tratando como vacío.")
                 ner_tags_reconstructed = []
                 num_entities_in_example = 0
            else:
                 ner_tags_reconstructed = [] # List[Dict]
                 if num_entities_in_example > 0:
                     for j in range(num_entities_in_example):
                         # Validar tipos individuales
                         if not all(isinstance(x, int) for x in [starts_list[j], ends_list[j], labels_list[j]]):
                              print(f"Advertencia: Tipo inesperado en datos de entidad reconstruida para ej:{i}, ent:{j}. Omitiendo entidad.")
                              continue
                         # Reconstruir el diccionario para la entidad j
                         # *** USANDO EL ACCESO CORRECTO A starts_list[j], etc. ***
                         ner_tags_reconstructed.append({
                             'start': starts_list[j], # Acceder al j-th elemento
                             'end': ends_list[j],     # Acceder al j-th elemento
                             'label': labels_list[j]  # Acceder al j-th elemento (ID)
                         })
        except KeyError as e:
             print(f"Error: Clave faltante '{e}' al acceder a diccionario de ner_tags para el índice {i}. Dict recibido: {examples['ner_tags'][i]}")
             ner_tags_reconstructed = []
        except IndexError as e:
             # Este error ahora no debería ocurrir con .get y validación de longitud
             print(f"Error: Índice fuera de rango al acceder a listas internas para el índice {i}: {e}")
             ner_tags_reconstructed = []
        except Exception as e: # Captura otros errores inesperados
             print(f"Error inesperado reconstruyendo ner_tags para índice {i}: {e}")
             ner_tags_reconstructed = []

        # -----------------------------------------------------------

        # Obtener mapeos y IDs de palabra para el ejemplo 'i'
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        offset_mapping = tokenized_inputs.offset_mapping[i]
        previous_word_idx = None
        # Inicializar etiquetas para este ejemplo con -100 (ignore index)
        label_ids_for_example = [-100] * len(word_ids)

        # Ordenar las entidades reconstruidas (ner_tags_reconstructed es List[Dict])
        sorted_ner_tags = []
        if ner_tags_reconstructed:
             try:
                 valid_tags_for_sorting = [tag for tag in ner_tags_reconstructed if isinstance(tag, dict) and 'start' in tag]
                 if len(valid_tags_for_sorting) != len(ner_tags_reconstructed):
                     print(f"Advertencia: Se filtraron tags inválidos antes de ordenar en índice {i}.")
                 sorted_ner_tags = sorted(valid_tags_for_sorting, key=lambda x: x['start'])
             except Exception as e:
                 print(f"Error al ordenar etiquetas reconstruidas para índice {i}: {e}")
                 sorted_ner_tags = ner_tags_reconstructed # Usar sin ordenar

        # --- Lógica de Alineación con B/I CORREGIDO (Sin cambios aquí) ---
        current_tag_idx = 0
        for token_idx, current_word_idx in enumerate(word_ids):

            if current_word_idx is None: # Token especial [CLS], [SEP]
                label_ids_for_example[token_idx] = -100
                previous_word_idx = None
                continue

            token_start, token_end = offset_mapping[token_idx]
            if token_end <= token_start: # Token de Padding [PAD]
                 label_ids_for_example[token_idx] = -100
                 previous_word_idx = current_word_idx
                 continue

            # Manejar subtokens ANTES de la lógica B/I principal
            is_subword = (current_word_idx == previous_word_idx)
            if is_subword:
                 if not LABEL_ALL_SUBWORDS:
                     label_ids_for_example[token_idx] = -100
                 else:
                     # Lógica para heredar etiqueta (B->I, I->I)
                     prev_label_id = label_ids_for_example[token_idx-1]
                     if prev_label_id != -100:
                         prev_label_str = id2label.get(prev_label_id, "O")
                         if prev_label_str != "O": # Si era B- o I-
                             base_label_str = prev_label_str.split('-')[-1]
                             i_label_str = f"I-{base_label_str}"
                             label_ids_for_example[token_idx] = label2id.get(i_label_str, -100)
                         else: # Si era O
                              label_ids_for_example[token_idx] = -100
                     else: # Si el anterior era -100
                          label_ids_for_example[token_idx] = -100
                 # Actualizar previous_word_idx y saltar el resto del bucle para subwords
                 previous_word_idx = current_word_idx
                 continue

            # --- Es el inicio de una nueva palabra (no subword) ---
            current_label_id = label2id["O"] # Default a 'O'
            current_base_label_str = None

            # Buscar la entidad que cubre este token
            for tag_idx in range(current_tag_idx, len(sorted_ner_tags)):
                tag = sorted_ner_tags[tag_idx]
                if not isinstance(tag, dict) or not all(k in tag for k in ['start', 'end', 'label']): continue

                if token_start >= tag['start'] and token_start < tag['end']:
                    try:
                        current_base_label_str = label_list[tag['label']] # 'NOMBRE_EMPRESA'
                    except (IndexError, KeyError): current_base_label_str = None
                    break # Encontramos la entidad para este token
                elif token_start >= tag['end']:
                    current_tag_idx += 1 # Ya pasamos esta entidad

            # Asignar B- o I- si el token cae dentro de una entidad
            if current_base_label_str is not None:
                # Mirar la etiqueta del token útil anterior (no -100)
                previous_token_label_id = label2id["O"] # Default si es el primer token útil
                for k in range(token_idx - 1, -1, -1):
                     if label_ids_for_example[k] != -100:
                          previous_token_label_id = label_ids_for_example[k]
                          break

                previous_token_label_str = id2label.get(previous_token_label_id, "O")

                # Comprobar si la etiqueta anterior era parte de la MISMA entidad
                is_continuation = (previous_token_label_str != "O" and
                                   previous_token_label_str.endswith(current_base_label_str))

                if is_continuation:
                    # Asignar I-LABEL
                    current_label_id = label2id.get(f"I-{current_base_label_str}", label2id["O"]) # Fallback O
                else:
                    # Asignar B-LABEL (inicio de entidad o entidad diferente)
                    current_label_id = label2id.get(f"B-{current_base_label_str}", label2id["O"]) # Fallback O

            label_ids_for_example[token_idx] = current_label_id
            # Actualizar previous_word_idx para la próxima iteración
            previous_word_idx = current_word_idx
        # --- Fin lógica de alineación ---

        all_aligned_labels.append(label_ids_for_example) # Añadir la lista procesada al resultado del lote

    # Devolver el diccionario con las etiquetas alineadas bajo la clave 'labels'
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs

print("Función de tokenización y alineación (ACCESO Y LÓGICA B/I CORREGIDOS) definida.")

In [ ]:
if dataset_hf:
    print("\nTokenizando y alineando etiquetas para todo el dataset...")
    try:
        tokenized_dataset = dataset_hf.map(
            tokenize_and_align_labels,
            batched=True, # Procesar en lotes
            remove_columns=dataset_hf.column_names # Eliminar columnas antiguas
        )
        print("Tokenización completada.")
        print("Ejemplo de datos procesados (primer ejemplo):")
        # Imprimir con manejo de error por si id2label falla
        try:
          print("Tokens:", tokenizer_ner.convert_ids_to_tokens(tokenized_dataset[0]['input_ids']))
          print("Labels:", [id2label.get(lbl_id, f"ID_{lbl_id}") for lbl_id in tokenized_dataset[0]['labels']])
        except Exception as e:
             print(f"Error al imprimir ejemplo procesado: {e}")
             print("Input IDs:", tokenized_dataset[0]['input_ids'])
             print("Label IDs:", tokenized_dataset[0]['labels'])

    except Exception as e:
        print(f"\nError durante el proceso de .map(): {e}")
        traceback.print_exc()
        raise RuntimeError("Fallo durante la tokenización/alineación.")
else:
    print("Error: dataset_hf no está definido. No se puede tokenizar.")
    raise RuntimeError("Dataset no disponible para tokenizar.")

Dividir el dataset

In [ ]:
if 'tokenized_dataset' in locals():
    # Dividir en entrenamiento y validación (ej. 90/10)
    train_val_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
    final_datasets = DatasetDict({
        'train': train_val_split['train'],
        'validation': train_val_split['test']
    })
    print("\nDatasets divididos para entrenamiento/validación:")
    print(final_datasets)
else:
    print("Error: tokenized_dataset no fue creado. No se puede dividir.")
    raise RuntimeError("Dataset tokenizado no disponible.")

 Cargar Modelo NER Pre-entrenado

In [ ]:
# --- Cargar Modelo NER ---
print(f"\nCargando modelo NER pre-entrenado: {MODELO_BERT_NER}")
try:
    model_ner = AutoModelForTokenClassification.from_pretrained(
        MODELO_BERT_NER,
        num_labels=num_labels_ner, # Número de etiquetas BIO
        id2label=id2label,       # Mapeo ID -> Label (para inferencia)
        label2id=label2id        # Mapeo Label -> ID (para inferencia)
    )
    print("Modelo NER cargado.")
    # --- Configurar Dispositivo --- (Hacerlo aquí antes del Trainer)
    device_ner = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_ner.to(device_ner)
    print(f"Modelo movido al dispositivo: {device_ner}")
    if device_ner.type == 'cpu':
       print("ADVERTENCIA: Entrenar en CPU será significativamente más lento.")

except Exception as e:
    print(f"Error al cargar el modelo NER {MODELO_BERT_NER}: {e}")
    raise RuntimeError("Fallo al cargar el modelo NER.")

Definir Argumentos de Entrenamiento

In [ ]:
# --- Configuración del Entrenamiento ---
print("\nConfigurando los argumentos de entrenamiento...")
training_args_ner = TrainingArguments(
    output_dir=OUTPUT_DIR_NER,
    num_train_epochs=EPOCHS_NER,
    per_device_train_batch_size=BATCH_SIZE_NER,
    per_device_eval_batch_size=BATCH_SIZE_NER,
    learning_rate=LEARNING_RATE_NER,
    weight_decay=0.01,
    eval_strategy="epoch", # Evaluar después de cada época
    save_strategy="epoch",       # Guardar después de cada época
    load_best_model_at_end=True, # Cargar el mejor modelo al final
    metric_for_best_model="eval_f1_weighted", # Usar F1-score de validación para decidir el mejor
    greater_is_better=True,
    logging_dir=f"{OUTPUT_DIR_NER}/logs",
    logging_steps=50, # Loggear cada 50 pasos
    save_total_limit=2, # Guardar solo los últimos 2 checkpoints + el mejor
    push_to_hub=False, # No subir al Hub de Hugging Face
    report_to = "none"
)
print("Argumentos de entrenamiento definidos.")

Definir Data Collator y Función de Métricas

In [ ]:
# --- Data Collator ---
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer_ner)
print("Data Collator definido.")

# --- Métrica de Evaluación (usando seqeval) ---
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2) # Obtener IDs predichos

    # Convertir IDs a etiquetas BIO strings, ignorando -100
    true_labels = [
        [id2label.get(l, 'O') for l in label if l != -100] # Usar 'O' si falta ID
        for label in labels
    ]
    true_predictions = [
        [id2label.get(p, 'O') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    try:
        # Calcular métricas usando seqeval
        report = classification_report(true_labels, true_predictions, output_dict=True, mode='strict', scheme=IOB2, zero_division=0)

        # Extraer métricas clave
        results = {
            # Usar macro avg para F1/precision/recall puede ser más informativo si hay desbalance
            "precision_macro": report["macro avg"]["precision"],
            "recall_macro": report["macro avg"]["recall"],
            "f1_macro": report["macro avg"]["f1-score"],
            "precision_weighted": report["weighted avg"]["precision"],
            "recall_weighted": report["weighted avg"]["recall"],
            "f1_weighted": report["weighted avg"]["f1-score"],
        }
    except Exception as e:
        print(f"\nError en compute_metrics: {e}")
        print("True Labels (muestra):", true_labels[:2])
        print("True Predictions (muestra):", true_predictions[:2])
        # Devolver F1=0 en caso de error para no romper el entrenamiento
        results = {"f1_macro": 0.0, "f1_weighted": 0.0}

    return results

print("Función compute_metrics definida.")

Inicializar Trainer

In [ ]:
# --- Inicializar Trainer ---
trainer = Trainer(
    model=model_ner,
    args=training_args_ner,
    train_dataset=final_datasets["train"],
    eval_dataset=final_datasets["validation"],
    tokenizer=tokenizer_ner,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
print("Trainer inicializado.")

Ejecutar Entrenamiento

In [31]:
# --- Iniciar Entrenamiento ---
print("\n--- Iniciando Entrenamiento NER ---")
try:
    train_result = trainer.train()
    print("--- Entrenamiento NER Completado ---")

    # Guardar métricas finales
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

except Exception as e:
    print(f"\nError durante el entrenamiento: {e}")
    traceback.print_exc()
    raise RuntimeError("Fallo durante el entrenamiento.")


--- Iniciando Entrenamiento NER ---


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Precision Weighted,Recall Weighted,F1 Weighted
1,0.006600,0.004224,0.988933,0.989170,0.988916,0.987663,0.988070,0.987719
2,0.001500,0.001584,0.996275,0.996867,0.996570,0.995796,0.996491,0.996143
3,0.000800,0.000114,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,0.000900,0.000060,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,0.000100,0.000043,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


--- Entrenamiento NER Completado ---
***** train metrics *****
  epoch                    =        5.0
  total_flos               =  3899933GF
  train_loss               =     0.0195
  train_runtime            = 0:20:23.49
  train_samples_per_second =     52.387
  train_steps_per_second   =      3.277


Guardar el modelo

In [ ]:
# --- Guardar el Mejor Modelo y Tokenizador ---
# El Trainer ya guarda el mejor modelo en output_dir si load_best_model_at_end=True
# Pero podemos forzar un guardado explícito del estado final si queremos.
print(f"\nGuardando el modelo final (mejor modelo cargado) en: {OUTPUT_DIR_NER}")
try:
    trainer.save_model(OUTPUT_DIR_NER)
    tokenizer_ner.save_pretrained(OUTPUT_DIR_NER)
    # El mapeo de etiquetas ya se guardó antes.
    print("¡Modelo NER y tokenizador guardados exitosamente!")
    print(f"Archivos guardados en {OUTPUT_DIR_NER}: {os.listdir(OUTPUT_DIR_NER)}")
except Exception as e:
    print(f"Error al guardar el modelo/tokenizador final: {e}")

Cargar el modelo

In [ ]:
# --- Cargar Modelo y Tokenizador Guardados para Inferencia ---
print(f"\n--- Cargando modelo NER guardado desde: {OUTPUT_DIR_NER} ---")

# Directorio donde se guardó el mejor modelo
model_load_path = OUTPUT_DIR_NER

# Re-cargar mapeos
map_file_path_ner = os.path.join(model_load_path, 'ner_label_mappings.json')
try:
    with open(map_file_path_ner, 'r', encoding='utf-8') as f:
        mappings_ner = json.load(f)
    id2label_cargado = {int(k): v for k, v in mappings_ner['id2label'].items()}
    label2id_cargado = mappings_ner['label2id']
    print("Mapeos de etiquetas NER cargados.")
except Exception as e:
    print(f"Error al cargar mapeos NER desde {map_file_path_ner}: {e}")
    raise RuntimeError("Fallo al cargar mapeos.")

# Cargar modelo y tokenizador
try:
    model_cargado_ner = AutoModelForTokenClassification.from_pretrained(model_load_path)
    tokenizer_cargado_ner = AutoTokenizer.from_pretrained(model_load_path)
    print(f"Modelo y tokenizador NER cargados exitosamente desde {model_load_path}.")
except Exception as e:
    print(f"Error al cargar modelo/tokenizador NER desde {model_load_path}: {e}")
    raise RuntimeError("Fallo al cargar modelo/tokenizador.")

# Configurar dispositivo y modo evaluación
# Determinar el dispositivo de nuevo por si se reinició el kernel
device_inf = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cargado_ner.to(device_inf)
model_cargado_ner.eval()
print(f"Modelo NER listo para inferencia en dispositivo: {device_inf}")

In [ ]:
# --- Función de Predicción NER ---
def predecir_ner(texto, modelo, tokenizer, device, id_to_label_map, max_len):
    """Predice entidades NER en un texto dado."""
    if not texto or not isinstance(texto, str) or texto.strip() == "":
        return [], "Texto inválido."

    # 1. Tokenizar y obtener offsets
    inputs = tokenizer(
        texto,
        return_tensors='pt',
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_offsets_mapping=True
    )
    offset_mapping = inputs.pop("offset_mapping").cpu().squeeze().tolist()
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 2. Inferencia
    with torch.no_grad():
        outputs = modelo(**inputs)

    # 3. Obtener predicciones (IDs)
    predictions = torch.argmax(outputs.logits, dim=-1).cpu().squeeze().tolist()

    # 4. Reconstruir entidades
    entities = []
    current_entity_tokens = []
    current_entity_label = None
    start_offset = -1

    for i, pred_id in enumerate(predictions):
        start_char, end_char = offset_mapping[i]

        # Ignorar tokens especiales/padding (offset 0,0 o fuera del texto original)
        if start_char == end_char or end_char == 0:
             # Si terminamos una entidad justo antes del padding/special token
            if current_entity_tokens:
                entities.append({
                    "text": texto[start_offset:current_entity_end_offset],
                    "label": current_entity_label,
                    "start": start_offset,
                    "end": current_entity_end_offset
                })
                current_entity_tokens = []
                current_entity_label = None
                start_offset = -1
            continue # Saltar token especial/padding

        label_name = id_to_label_map.get(pred_id, "O")

        if label_name.startswith("B-"):
            # Si había una entidad abierta, la guardamos
            if current_entity_tokens:
                 entities.append({
                    "text": texto[start_offset:current_entity_end_offset],
                    "label": current_entity_label,
                    "start": start_offset,
                    "end": current_entity_end_offset
                 })
            # Empezamos una nueva entidad
            current_entity_tokens = [(start_char, end_char)]
            current_entity_label = label_name[2:]
            start_offset = start_char
            current_entity_end_offset = end_char # Guardar el end offset del último token

        elif label_name.startswith("I-"):
            # Si estamos continuando una entidad del mismo tipo
            if current_entity_tokens and current_entity_label == label_name[2:]:
                current_entity_tokens.append((start_char, end_char))
                current_entity_end_offset = end_char # Actualizar el end offset
            # Si es I- pero no había B- o el tipo no coincide, la cerramos y empezamos nueva?
            # Opción más segura: cerrar la anterior (si existe) e ignorar esta I- 'huérfana'.
            elif current_entity_tokens:
                 entities.append({
                    "text": texto[start_offset:current_entity_end_offset],
                    "label": current_entity_label,
                    "start": start_offset,
                    "end": current_entity_end_offset
                 })
                 current_entity_tokens = []
                 current_entity_label = None
                 start_offset = -1

        elif label_name == "O":
            # Si encontramos 'O' y había una entidad abierta, la cerramos
            if current_entity_tokens:
                entities.append({
                    "text": texto[start_offset:current_entity_end_offset],
                    "label": current_entity_label,
                    "start": start_offset,
                    "end": current_entity_end_offset
                })
                current_entity_tokens = []
                current_entity_label = None
                start_offset = -1

    # Asegurarse de guardar la última entidad si el texto termina con ella
    if current_entity_tokens:
        entities.append({
            "text": texto[start_offset:current_entity_end_offset],
            "label": current_entity_label,
            "start": start_offset,
            "end": current_entity_end_offset
        })

    return entities, texto

print("Función de predicción NER definida.")

CHAT

In [ ]:
# --- Bucle de Chat Simple NER ---
print("\n--- Chat Simple NER ---")
print("Escribe una frase para extraer entidades o 'salir' para terminar.")

while True:
    try:
        frase_usuario = input("Tú: ")
        if frase_usuario.lower() in ['salir', 'exit', 'quit', 'terminar']:
            print("Chatbot NER: ¡Adiós!")
            break

        entidades_encontradas, texto_original = predecir_ner(
            frase_usuario,
            model_cargado_ner,
            tokenizer_cargado_ner,
            device_inf, # Usar el dispositivo determinado para inferencia
            id2label_cargado,
            MAX_LEN_NER
        )

        print("\nChatbot NER:")
        if entidades_encontradas:
            print("  Texto Original:", texto_original)
            print("  --- Entidades Encontradas ---")
            for ent in entidades_encontradas:
                print(f"    Texto : '{ent['text']}'")
                print(f"    Label : {ent['label']}")
                print(f"    Pos   : ({ent['start']}, {ent['end']})")
                print("-" * 20) # Separador
        elif texto_original != "Texto inválido.":
            print("  No se encontraron entidades conocidas en la frase.")
        else:
            print(f"  Error: {texto_original}") # Mensaje si el texto fue inválido
        print("\n") # Espacio extra

    except KeyboardInterrupt:
        print("\nChatbot NER: ¡Adiós!")
        break
    except Exception as e:
        print(f"\nChatbot NER: Ocurrió un error inesperado: {e}")
        traceback.print_exc()